# Spotify Recommendation Algorithm 

The purpose of this project is to design a music recommendation algorithm that fits closer to my taste. For a detailed writeup please visit this doc: https://docs.google.com/document/d/1XB79YVOwlACy4Q7-vdG1uTmbOqfpizOogoq2l9OrFM0/edit?usp=sharing

Thanks for checking out this project!

### Create datasets from Spotify playlists utilizing the Spotipy API

In [1]:
import spotipy
import pandas as pd
import numpy as np

In [2]:
# spotipy connectivity setup 
CLIENT_ID = 'bdd43f9c189349e582fdc5fa6a3f0891'
CLIENT_SECRET = '541e1a33bf23481aa51d97043c9516e9'
client_credentials_manager = spotipy.oauth2.SpotifyClientCredentials(client_id=CLIENT_ID, client_secret=CLIENT_SECRET)
sp = spotipy.Spotify(client_credentials_manager = client_credentials_manager)

In [3]:
# stores and convert spotify playlist data into a df 
# the code was altered from this website
# https://towardsdatascience.com/how-to-create-large-music-datasets-using-spotipy-40e7242cc6a6
def analyze_playlist(creator, playlist_id):
    
    # Create empty dataframe
    playlist_features_list = ["artist","album","track_name",  "track_id","danceability","energy","key","loudness","mode", "speechiness","instrumentalness","liveness","valence","tempo", "duration_ms","time_signature"]
    
    playlist_df = pd.DataFrame(columns = playlist_features_list)
    
    # Loop through every track in the playlist, extract features and append the features to the playlist df
    # take out 'track' level dict if it's my own play list 
    playlist = sp.user_playlist_tracks(creator, playlist_id)['tracks']["items"]
    for track in playlist:
        # Create empty dict
        playlist_features = {}
        # Get metadata
        playlist_features["artist"] = track["track"]["album"]["artists"][0]["name"]
        playlist_features["album"] = track["track"]["album"]["name"]
        playlist_features["track_name"] = track["track"]["name"]
        playlist_features["track_id"] = track["track"]["id"]
        
        # Get audio features
        audio_features = sp.audio_features(playlist_features["track_id"])[0]
        for feature in playlist_features_list[4:]:
            playlist_features[feature] = audio_features[feature]
        
        # Concat the dfs
        track_df = pd.DataFrame(playlist_features, index = [0])
        playlist_df = pd.concat([playlist_df, track_df], ignore_index = True)
        
    return playlist_df

In [195]:
# this df includes songs in my current playlists and songs that I dislike
df = pd.read_csv('track data.csv')

### Data Wrangling &  Exploration

In [196]:
df.isnull().sum()

artist              0
album               0
track_name          0
track_id            0
danceability        0
energy              0
key                 0
loudness            0
mode                0
speechiness         0
instrumentalness    0
liveness            0
valence             0
tempo               0
duration_ms         0
time_signature      0
label               0
dtype: int64

In [197]:
df.describe()

,danceability,energy,key,loudness,mode,speechiness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature,label
count,233.000000,233.000000,233.000000,233.000000,233.000000,233.000000,233.000000,233.000000,233.000000,233.000000,233.000000,233.000000,233.000000
mean,0.620039,0.541988,5.038627,-8.583193,0.669528,0.067315,0.113319,0.165550,0.470689,118.453966,218081.008584,3.905579,0.553648
std,0.142777,0.213900,3.565068,3.251491,0.471396,0.067930,0.237950,0.128468,0.238778,29.910018,54586.865616,0.334269,0.498184
min,0.192000,0.052900,0.000000,-23.465000,0.000000,0.023200,0.000000,0.026400,0.051700,60.496000,98544.000000,1.000000,0.000000
25%,0.532000,0.381000,2.000000,-10.381000,0.000000,0.031500,0.000003,0.094400,0.262000,93.842000,181893.000000,4.000000,0.000000
50%,0.624000,0.537000,5.000000,-8.216000,1.000000,0.039300,0.000890,0.112000,0.472000,117.401000,209999.000000,4.000000,1.000000
75%,0.739000,0.717000,8.000000,-6.118000,1.000000,0.068800,0.047100,0.192000,0.633000,139.980000,245426.000000,4.000000,1.000000
max,0.887000,0.968000,11.000000,-2.724000,1.000000,0.440000,0.938000,0.954000,0.967000,203.908000,515865.000000,4.000000,1.000000


## Model Training

### SVM

"Linear kernel" worked the best with the highest score (mean accuracy). I also dropped the loudness and duration_ms columns to increase the accurracy. 

In [73]:
from sklearn import svm
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report

In [200]:
x = df2.loc[:,'danceability':'time_signature']
y = df2['label']

In [201]:
svm_model = svm.SVC(kernel = 'linear').fit(x, y)
svm_model.score(x, y)

0.575107296137339

In [109]:
# check the coefficients from the data 
svm_model.coef_

array([[-2.63222673e+00, -9.65268823e+00,  5.78623479e+01,
        -1.48507967e+02,  6.00000000e+00, -8.60379321e-01,
         2.26166693e+00,  1.07576171e-01, -5.47104210e+00,
        -1.03275776e+01,  7.48187154e-02, -5.00000000e+00]])

In [204]:
# drop irrrelevant coefficients 
x = x.drop(columns = ['loudness','duration_ms'])
svm_model = svm.SVC(kernel = 'linear').fit(x, y)
svm_model.score(x, y)

0.7553648068669528

### K Nearest Neighbors

In [114]:
from sklearn.neighbors import KNeighborsRegressor

In [211]:
# also tried n_neighbors =  2, 3, & 4
knn_model = KNeighborsRegressor(n_neighbors=5).fit(x,y)
knn_model.score(x,y)

0.18651460942158604

### Logistic Regression

In [205]:
from sklearn.linear_model import LogisticRegression

In [206]:
# tried solver = 'liblinear'
log_model = LogisticRegression(max_iter = 200, solver = 'lbfgs').fit(x,y)
log_model.score(x,y)

0.7510729613733905

### Neural Network

In [148]:
from sklearn.neural_network import MLPClassifier

In [237]:
# continued using lbfgs 
mlp = MLPClassifier(hidden_layer_sizes=(10,10),max_iter=500, solver='lbfgs', random_state = 1).fit(x,y)
mlp.score(x,y)

0.7939914163090128

### Model Results
Here I output the results to Excel files then manually classified the songs. Again for the detailed write-up, please visit this page. 
https://docs.google.com/document/d/1XB79YVOwlACy4Q7-vdG1uTmbOqfpizOogoq2l9OrFM0/edit?usp=sharing

Note that I did not export the results for "Discover Weekly" since the playlist only contains 30 songs and the classification results can be observed fairly easily (please refer to the dataframe in each model).

In [250]:
# outputs prediction results based on the model and the dataset
def pred_results(df, model):
    x = df.loc[:,'danceability':'time_signature']
    x = x.drop(columns = ['loudness','duration_ms'])
    x_pred = model.predict(x)
    df['pred'] = x_pred
    return df

### SVM

Please note that Spotify playlist tracks get updated from time to time. Therefore the metrics maybe inconsistent by the time you run this code. 

In [ ]:
# import "Discover Weekly" playlist
df_discover = analyze_playlist("spotify","37i9dQZEVXcDjNKMX0xagW?si=b0f662cb410f4082")

In [244]:
# Get the results from svm model 
# three misclassification - Cloud9(dislike), Silence(dislike), and What a Beautiful Night(like)
discover_weekly_svm = pred_results(df_discover, svm_model)
discover_weekly_svm

,artist,album,track_name,track_id,danceability,energy,key,loudness,mode,speechiness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature,pred
0,柯智棠,The Joy Of Sorrow,The Joy Of Sorrow,78oPAwgoMFL9QxnueTuNwy,0.462,0.1200,5,-16.798,1,0.0396,0.000019,0.1430,0.232,73.228,207435,4,1
1,Tomoyo Harada,Candle Lights,ラヴ・ミー・テンダー - Haruomi Hosono Rework,1U14MlCuOIk5XDhvJD8Ct0,0.456,0.1680,9,-13.825,1,0.0294,0.001350,0.1060,0.066,137.711,210453,4,1
2,Tokyo Incidents,大人,黄昏泣き,17OAQmU919MnDrmb8nDv7U,0.319,0.3850,2,-11.213,0,0.0332,0.923000,0.1060,0.239,73.611,208267,4,1
3,HONNE,nswy: dream edits,free love - dream edit,3HAsf0o0TJY9WL4zKCzE3u,0.670,0.2200,8,-10.214,1,0.0422,0.000076,0.1110,0.139,119.604,249945,4,1
4,Faye Wong,王菲97,你快樂所以我快樂,31kKBUeeCGuMt9fPxDybde,0.676,0.3070,8,-12.919,0,0.0279,0.002840,0.1230,0.163,110.857,258427,4,1
5,Hanare Gumi,帰ってから、歌いたくなってもいいようにと思ったのだ。,ねむるのまち~Tidur Tidur~,2CXT8P6nv9BfKMn4xSMU8V,0.510,0.0328,7,-22.611,1,0.0390,0.083000,0.2050,0.428,131.640,310520,3,1
6,Tanya Chua,若你碰到他,若你碰到他,74vamOEacSToHKKvY076mq,0.372,0.2110,11,-9.255,1,0.0294,0.000000,0.0921,0.225,137.489,333680,3,1
7,中村佳穂,AINOU,忘れっぽい天使,3vVOzWA6SNSGXNYeg4LnYN,0.443,0.0650,0,-14.520,1,0.0601,0.000000,0.1110,0.287,71.770,235506,4,1
8,NELL,Let’s Part,Time Walking on Memories,6XkrfYmgPGSvgufoivTQgj,0.399,0.3310,10,-9.221,1,0.0478,0.000001,0.1100,0.193,171.208,389853,4,1
9,Anni Hung,洪安妮 我喜歡你,我喜歡你,08cKcaqRuwzfEYZac1RKAF,0.407,0.2740,1,-12.247,1,0.0392,0.000358,0.1190,0.235,134.577,246924,4,1


In [212]:
# global top 50 playist  
df_global = analyze_playlist("spotify","37i9dQZEVXbMDoHDwVN2tF?si=5ec773049fe544c9")

In [259]:
# exporting svm results to csv
glob_svm = pred_results(df_global, trainedsvm)
glob_results.to_csv('Global Results.csv', index_label = None)

In [222]:
#df_indie = analyze_playlist("spotify","37i9dQZF1DX2Nc3B70tvx0?si=016b1ff54ad04b4e")
indie_svm = pred_results(df_indie, svm_model)
indie_svm.to_csv('Indie Results.csv', index_label = None)

### Logistic Regression Results

In [253]:
discover_weekly_log = pred_results(df_discover, log_model)
discover_weekly_log

,artist,album,track_name,track_id,danceability,energy,key,loudness,mode,speechiness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature,pred
0,柯智棠,The Joy Of Sorrow,The Joy Of Sorrow,78oPAwgoMFL9QxnueTuNwy,0.462,0.1200,5,-16.798,1,0.0396,0.000019,0.1430,0.232,73.228,207435,4,1
1,Tomoyo Harada,Candle Lights,ラヴ・ミー・テンダー - Haruomi Hosono Rework,1U14MlCuOIk5XDhvJD8Ct0,0.456,0.1680,9,-13.825,1,0.0294,0.001350,0.1060,0.066,137.711,210453,4,1
2,Tokyo Incidents,大人,黄昏泣き,17OAQmU919MnDrmb8nDv7U,0.319,0.3850,2,-11.213,0,0.0332,0.923000,0.1060,0.239,73.611,208267,4,1
3,HONNE,nswy: dream edits,free love - dream edit,3HAsf0o0TJY9WL4zKCzE3u,0.670,0.2200,8,-10.214,1,0.0422,0.000076,0.1110,0.139,119.604,249945,4,1
4,Faye Wong,王菲97,你快樂所以我快樂,31kKBUeeCGuMt9fPxDybde,0.676,0.3070,8,-12.919,0,0.0279,0.002840,0.1230,0.163,110.857,258427,4,1
5,Hanare Gumi,帰ってから、歌いたくなってもいいようにと思ったのだ。,ねむるのまち~Tidur Tidur~,2CXT8P6nv9BfKMn4xSMU8V,0.510,0.0328,7,-22.611,1,0.0390,0.083000,0.2050,0.428,131.640,310520,3,1
6,Tanya Chua,若你碰到他,若你碰到他,74vamOEacSToHKKvY076mq,0.372,0.2110,11,-9.255,1,0.0294,0.000000,0.0921,0.225,137.489,333680,3,1
7,中村佳穂,AINOU,忘れっぽい天使,3vVOzWA6SNSGXNYeg4LnYN,0.443,0.0650,0,-14.520,1,0.0601,0.000000,0.1110,0.287,71.770,235506,4,1
8,NELL,Let’s Part,Time Walking on Memories,6XkrfYmgPGSvgufoivTQgj,0.399,0.3310,10,-9.221,1,0.0478,0.000001,0.1100,0.193,171.208,389853,4,1
9,Anni Hung,洪安妮 我喜歡你,我喜歡你,08cKcaqRuwzfEYZac1RKAF,0.407,0.2740,1,-12.247,1,0.0392,0.000358,0.1190,0.235,134.577,246924,4,1


In [254]:
# output top 50 global playlist to Excel for further calculation  
glob_log = pred_results(df_global, log_model)
glob_log.to_csv('Global Log.csv', index = False)

In [256]:
# output ultimate indie to Excel
indie_log = pred_results(df_indie, log_model)
indie_log.to_csv('Indie Log.csv', index_label = None)

### Neural Network Results

In [252]:
discover_weekly_nn = pred_results(df_discover, mlp)
discover_weekly_nn

,artist,album,track_name,track_id,danceability,energy,key,loudness,mode,speechiness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature,pred
0,柯智棠,The Joy Of Sorrow,The Joy Of Sorrow,78oPAwgoMFL9QxnueTuNwy,0.462,0.1200,5,-16.798,1,0.0396,0.000019,0.1430,0.232,73.228,207435,4,1
1,Tomoyo Harada,Candle Lights,ラヴ・ミー・テンダー - Haruomi Hosono Rework,1U14MlCuOIk5XDhvJD8Ct0,0.456,0.1680,9,-13.825,1,0.0294,0.001350,0.1060,0.066,137.711,210453,4,1
2,Tokyo Incidents,大人,黄昏泣き,17OAQmU919MnDrmb8nDv7U,0.319,0.3850,2,-11.213,0,0.0332,0.923000,0.1060,0.239,73.611,208267,4,1
3,HONNE,nswy: dream edits,free love - dream edit,3HAsf0o0TJY9WL4zKCzE3u,0.670,0.2200,8,-10.214,1,0.0422,0.000076,0.1110,0.139,119.604,249945,4,1
4,Faye Wong,王菲97,你快樂所以我快樂,31kKBUeeCGuMt9fPxDybde,0.676,0.3070,8,-12.919,0,0.0279,0.002840,0.1230,0.163,110.857,258427,4,1
5,Hanare Gumi,帰ってから、歌いたくなってもいいようにと思ったのだ。,ねむるのまち~Tidur Tidur~,2CXT8P6nv9BfKMn4xSMU8V,0.510,0.0328,7,-22.611,1,0.0390,0.083000,0.2050,0.428,131.640,310520,3,1
6,Tanya Chua,若你碰到他,若你碰到他,74vamOEacSToHKKvY076mq,0.372,0.2110,11,-9.255,1,0.0294,0.000000,0.0921,0.225,137.489,333680,3,1
7,中村佳穂,AINOU,忘れっぽい天使,3vVOzWA6SNSGXNYeg4LnYN,0.443,0.0650,0,-14.520,1,0.0601,0.000000,0.1110,0.287,71.770,235506,4,1
8,NELL,Let’s Part,Time Walking on Memories,6XkrfYmgPGSvgufoivTQgj,0.399,0.3310,10,-9.221,1,0.0478,0.000001,0.1100,0.193,171.208,389853,4,1
9,Anni Hung,洪安妮 我喜歡你,我喜歡你,08cKcaqRuwzfEYZac1RKAF,0.407,0.2740,1,-12.247,1,0.0392,0.000358,0.1190,0.235,134.577,246924,4,1


In [255]:
# output top 50 global playlist to Excel  
glob_nn = pred_results(df_global, mlp)
glob_nn.to_csv('Global NN.csv', index = False)

In [257]:
# output ultimate indie to excel
indie_nn = pred_results(df_indie, mlp)
indie_nn.to_csv('Indie NN.csv', index_label = None)